In [5]:
from langchain.llms import GooglePalm

api_key = " " 

llm = GooglePalm(google_api_key=api_key, temperature=0.1)

In [6]:
poem = llm("Write a 4 line poem of my love for samosa")
print(poem)

**Oh, samosa, my love**
**You are so delicious, so flaky**
**With your filling of potatoes and peas**
**I could eat you every day**


In [7]:
from langchain.chains import RetrievalQA


from langchain.embeddings import GooglePalmEmbeddings
from langchain.llms import GooglePalm

In [12]:
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path='Q&A_chatbot.csv', source_column="prompt")

# Store the loaded data in the 'data' variable
data = loader.load()

In [37]:
from langchain_community.embeddings import HuggingFaceHubEmbeddings
embeddings = HuggingFaceHubEmbeddings()
query_result = embeddings.embed_query("What services do you offer")

In [38]:
len(query_result)

768

In [39]:
query_result[:5]

[0.05690343677997589,
 -0.01410953514277935,
 -0.026467522606253624,
 -0.004356516059488058,
 0.017473848536610603]

In [40]:
from langchain.vectorstores import FAISS

# Create a FAISS instance for vector database from 'data'
vectordb = FAISS.from_documents(documents=data,
                                 embedding=embeddings)

# Create a retriever for querying the vector database
retriever = vectordb.as_retriever(score_threshold = 0.7)

In [41]:
rdocs = retriever.get_relevant_documents("What kind of services do you offer?")
rdocs

[Document(page_content='prompt: What specific services does your marketing agency provide?\nresponse: We are providing services in the domain of marketing specialized in social media marketing, digital strategy, content marketing, personal branding and video production.', metadata={'source': 'What specific services does your marketing agency provide?', 'row': 0}),
 Document(page_content='prompt: Which type of services do you offer in content marketing\nresponse: We provide social media content, blog content that will align with your business goals.', metadata={'source': 'Which type of services do you offer in content marketing', 'row': 11}),
 Document(page_content='prompt: In content marketing which type of content service you are offering?\nresponse: We provide social media content and website content service.', metadata={'source': 'In content marketing which type of content service you are offering?', 'row': 10}),
 Document(page_content='prompt: What types of content do you recommend

In [42]:
from langchain.prompts import PromptTemplate

prompt_template = """Given the following context and a question, generate an answer based on this context only.
In the answer try to provide as much text as possible from "response" section in the source document context without making much changes.
If the answer is not found in the context, kindly state "I don't know." Don't try to make up an answer.

CONTEXT: {context}

QUESTION: {question}"""


PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}


from langchain.chains import RetrievalQA

chain = RetrievalQA.from_chain_type(llm=llm,
                            chain_type="stuff",
                            retriever=retriever,
                            input_key="query",
                            return_source_documents=True,
                            chain_type_kwargs=chain_type_kwargs)

In [43]:
chain('What kind of sevices do you provide?')

{'query': 'What kind of sevices do you provide?',
 'result': 'We are providing services in the domain of marketing specialized in social media marketing, digital strategy, content marketing, personal branding and video production.',
 'source_documents': [Document(page_content='prompt: What specific services does your marketing agency provide?\nresponse: We are providing services in the domain of marketing specialized in social media marketing, digital strategy, content marketing, personal branding and video production.', metadata={'source': 'What specific services does your marketing agency provide?', 'row': 0}),
  Document(page_content='prompt: Which type of services do you offer in content marketing\nresponse: We provide social media content, blog content that will align with your business goals.', metadata={'source': 'Which type of services do you offer in content marketing', 'row': 11}),
  Document(page_content='prompt: In content marketing which type of content service you are off

In [45]:
chain('Do you provide social media marketing?')

{'query': 'Do you provide social media marketing?',
 'result': 'We are providing all type of social media services in customized way.',
 'source_documents': [Document(page_content='prompt: Hi I want to inquire about your social media marketing service. can you provide overview of details?\nresponse: We are providing all type of social media services in customized way.', metadata={'source': 'Hi I want to inquire about your social media marketing service. can you provide overview of details?', 'row': 6}),
  Document(page_content="prompt: Can you share examples of successful social media campaigns you've managed for similar clients?\nresponse: we are startup and we have launched our services in 2024 . We can ensure the quality work that will align for your business goals . This is where you can trust on us", metadata={'source': "Can you share examples of successful social media campaigns you've managed for similar clients?", 'row': 3}),
  Document(page_content='prompt: Which type of servi